In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Summarization of Large Documents using LangChain 🦜🔗

> **NOTE:** This notebook uses the PaLM generative model, which will reach its [discontinuation date in October 2024](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text#model_versions). Please refer to [this updated notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/document-processing/document_processing.ipynb) for a version which uses the latest Gemini model.

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) |

## Overview

Text summarization is an NLP task that creates a concise and informative summary of a longer text. LLMs can be used to create summaries of news articles, research papers, technical documents, and other types of text.

Summarizing large documents can be challenging. To create summaries, you need to apply summarization strategies to your indexed documents. You have already seen some of these strategies in the previous notebooks. If you haven't completed it, it is recommended to do so to have a basic understanding of how to summarize large documents.

In this notebook, you will use LangChain, a framework for developing LLM applications, to apply some summarization strategies. The notebook covers several examples of how to summarize large documents.

### Objective

In this tutorial, you learn how to use LangChain with PaLM API to summarize large documents by working through the following examples:

- Stuffing method
- MapReduce method
- Refine method

### Costs

This tutorial uses billable components of Google Cloud:
- Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK & Other dependencies

In [ ]:
!sudo apt -y -qq install tesseract-ocr
!sudo apt -y -qq install libtesseract-dev
!sudo apt-get -y -qq install poppler-utils #required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""

%pip install {USER} --upgrade pytesseract pypdf PyPDF2 textract langchain transformers google-cloud-aiplatform --quiet

**Colab only**: Run the following cell to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticating your notebook environment

- If you are using **Colab** to run this notebook, run the cell below and continue.
- If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

**Colab only:** Run the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.

In [1]:
!pip show PyPDFLoader

In [2]:
from pathlib import Path as p
import urllib
import warnings

from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyMuPDFLoader
from langchain.llms import Ollama
import pandas as pd

warnings.filterwarnings("ignore")

### Import models

You load the pre-trained text generation model called `text-bison`.

In [3]:
base_url = "http://localhost:11434"
model = "llama3.2:3b"

llm = Ollama(base_url=base_url,
                 model=model,
                 temperature=0.1,
                 num_predict=256
                 )

## Summarization with Large Documents

### Preparing data files

To begin, you will need to download a few files that are required for the summarizing tasks below.

### Extract text from the PDF

You use an `PdfReader` to extract the text from our scanned documents.

In [ ]:
docs = []
loader = PyMuPDFLoader("docs\\health supplements\\2. Nutraceuticals research.pdf")
pages = loader.load_and_split()
print(pages[3].page_content)

Table 1
Some deﬁnitions for food-derived products
Food supplement (United States
Government Ofﬁce, 1994)
A product (other than tobacco) in the form of a
capsule, powder, softgel or gelcap intended to
supplement the diet to enhance health that
bears or contains one or more of the
following dietary ingredients: a vitamin,
mineral, amino acid, or other botanical
or dietary substance.
United States Food and Drug Administration
(FDA). Dietary Supplement Health and Education
Act (DSHEA). U.S. Department of Health and
Human Services. 1994. United States. Public
Law 103–417, available at FDA Website:
http://www.fda.gov. [42]
Food supplement (EU, 2002)
Food product whose purpose is to
supplement the normal diet and which
consists of a concentrated source of nutrients
or other substances with nutritional effects
or physiological, single or in combination,
marketed in dosed formulations, such as capsules,
tablets, tablets or pills, designed to be taken
in small individual quantities measured.
EU 

## Method 1: Stuffing

Stuffing is the simplest method to pass data to a language model. It "stuffs" text into the prompt as context in a way that all of the relevant information can be processed by the model to get what you want.

In LangChain, you can use `StuffDocumentsChain` as part of the `load_summarize_chain` method. What you need to do is setting `stuff` as `chain_type` of your chain.

### Prompt design with `Stuffing` chain

In [5]:
prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

### Retrying
Initiate a chain using `stuff` method and process three pages document.

In [6]:
stuff_chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

In [7]:
three_pages = pages[:3]

In [8]:
three_pages

[Document(metadata={'source': 'docs\\health supplements\\2. Nutraceuticals research.pdf', 'file_path': 'docs\\health supplements\\2. Nutraceuticals research.pdf', 'page': 0, 'total_pages': 10, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113830Z', 'trapped': ''}, page_content='REVIEW\nNutraceuticals: opening the debate for a\nregulatory framework\nCorrespondence Ettore Novellino, Department of Pharmacy, Via D. Montesano, 49 - 80131 Naples, Italy. Tel.: +39 8167 9934;\nFax: +39 8167 8107; E-mail: ettore.novellino@unina.it\nReceived 13 April 2017; Revised 29 November 2017; Accepted 30 November 2017\nAntonello Santini1\n, Silvia Miriam Cammarata2, Giacomo Capone2, Angela Ianaro1, Gian Carlo Tenore1,\nLuca Pani3 and Ettore Novellino1\n1Department of Pharmacy, University of Napoli Federico II, Via D. Montesano, 49 -80131 Naples, Italy, 2AIFA - Agenzia Italiana del Farmaco, Via 

In [9]:
try:
    print(stuff_chain.run(three_pages))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

Here is a bullet point summary of the article:

**Introduction**

* The concept of nutraceuticals is not well-defined and lacks regulation
* There is a need for a clear understanding of nutraceuticals, their safety, efficacy, and mechanism of action

**Challenges in assessing nutraceuticals**

* Lack of shared regulations regarding nutraceuticals as pharma-foods
* Difficulty in distinguishing nutraceuticals from other food-derived products
* Need for more information on safety and efficacy based on clinical studies

**Current regulatory framework**

* The European Food Safety Authority (EFSA) is responsible for providing scientific advice on the beneficial health effects and associated risks of food intake
* The EFSA mission includes risk assessment, risk evaluation, and communication with stakeholders
* National authorities propose legislation and regulation in response to EFSA's recommendations

**Future directions**

* A different approach for nutraceutical concept assessment, use, 

As you can see, with the `stuff` method, you can summarize the entire document content with a single API call passing all data at once.

Depending on the context length of LLM, the `stuff` method would not work as it result in a prompt larger than the context length.

In [10]:
try:
    print(stuff_chain.run(pages))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

Here is a bullet point summary of the text:

**Citrinin and Red Yeast Rice (RYR)**

* Citrinin is a toxic metabolite found in RYR supplements
* The maximum acceptable level of citrinin was set at 2000 μg kg-1 by the German Federal Institute for Drugs and Medical Devices
* Analysis of citrinin in RYR using an immuno-affinity column enabled accurate detection with high sensitivity (LOQ: 10 μg kg-1, LOD: 3 μg kg-1)

**Regulatory Issues**

* The European Food Safety Authority (EFSA) has set a limit of monacolin K from RYR at 10 mg day-1 for the maintenance of normal blood levels
* However, this value is considered acceptable in many European countries despite potential health risks associated with high doses
* The US FDA issued a warning on RYR products in 2007 due to concerns over monacolin K content and potential health risks

**Safety Concerns**

* High values of monacolin K in RYR products may be considered with caution due to possible variability in content
* Dose inconsistency and sa

As expected, the code returns the expection message.

### Considerations

The `stuffing` method is a way to summarize text by feeding the entire document to a large language model (LLM) in a single call. This method has both pros and cons.

The stuffing method only requires a single call to the LLM, which can be faster than other methods that require multiple calls. When summarizing text, the LLM has access to all the data at once, which can result in a better summary.

But, LLMs have a context length, which is the maximum number of tokens that can be processed in a single call. If the document is longer than the context length, the stuffing method will not work. Also the stuffing method is not suitable for summarizing large documents, as it can be slow and may not produce a good summary.

Let's explore other approaches to help deal with having longer text than context length limit of LLMs.

## Method 2: MapReduce

The `MapReduce` method implements a multi-stage summarization. It is a technique for summarizing large pieces of text by first summarizing smaller chunks of text and then combining those summaries into a single summary.

In LangChain, you can use `MapReduceDocumentsChain` as part of the `load_summarize_chain` method. What you need to do is setting `map_reduce` as `chain_type` of your chain.

### Prompt design with `MapReduce` chain

In our example, you have a 32-page document that you need to summarize.

With LangChain, the `map_reduce` chain breaks the document down into 1024 token chunks max. Then it runs the initial prompt you define on each chunk to generate a summary of that chunk. In the example below, you use the following first stage or map prompt.

```Write a concise summary of the following text delimited by triple backquotes. Return your response in bullet points which covers the key points of the text.
'''{text}'''. BULLET POINT SUMMARY:```

Once summaries for all of the chunks are generated, it runs a different prompt to combine those summaries into a single summary. In the example below, you use the following second stage or combine prompt.

```Write a summary of the entire document that includes the main points from all of the individual summaries.```

In [11]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

### Generate summaries using MapReduce method

After defining prompts, you initialize the associated `map_reduce_chain`.

In [12]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

Then, you generate summaries using the chain. Notice that LangChain use a tokenizer (from transformer library) with 1024 token limit by default.

In [14]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (4740 > 1024). Running this sequence through the model will result in indexing errors


After summaries are generated, you can validate them by organize input documents and associated output in a Pandas DataFrame.

In [15]:
final_mp_data = []
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

In [16]:
pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,2. Nutraceuticals research,.pdf,0,REVIEW\nNutraceuticals: opening the debate for...,Here is a summary of the text with the main po...
1,2. Nutraceuticals research,.pdf,1,Introduction\nThere is a growing overlap betwe...,Here is a summary of the text with the main po...
2,2. Nutraceuticals research,.pdf,1,by clinical studies; (v) an evaluation of poss...,"Here is a summary of the text, including the m..."
3,2. Nutraceuticals research,.pdf,2,Table 1\nSome deﬁnitions for food-derived prod...,"Here is a summary of the text, including the m..."
4,2. Nutraceuticals research,.pdf,2,(FUFOSE). Scientiﬁc concepts of\nfunctional fo...,Here is a summary of the provided text:\n\nThe...


In [17]:
index = 3
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

[Context]
Table 1
Some deﬁnitions for food-derived products
Food supplement (United States
Government Ofﬁce, 1994)
A product (other than tobacco) in the form of a
capsule, powder, softgel or gelcap intended to
supplement the diet to enhance health that
bears or contains one or more of the
following dietary ingredients: a vitamin,
mineral, amino acid, or other botanical
or dietary substance.
United States Food and Drug Administration
(FDA). Dietary Supplement Health and Education
Act (DSHEA). U.S. Department of Health and
Human Services. 1994. United States. Public
Law 103–417, available at FDA Website:
http://www.fda.gov. [42]
Food supplement (EU, 2002)
Food product whose purpose is to
supplement the normal diet and which
consists of a concentrated source of nutrients
or other substances with nutritional effects
or physiological, single or in combination,
marketed in dosed formulations, such as capsules,
tablets, tablets or pills, designed to be taken
in small individual quantities mea

### Considerations

With `MapReduce` method, the model is able to summarize a large paper by overcoming the context limit of `Stuffing` method with parallel processing.

However, the `MapReduce` requires multiple calls to the model and potentially losing context between pages.

To deal this challenge, you can try another method to summarize multiple pages at a time.

## Method 3: Refine

The Refine method is an alternative method to deal with large document summarization. It works by first running an initial prompt on a small chunk of data, generating some output. Then, for each subsequent document, the output from the previous document is passed in along with the new document, and the LLM is asked to refine the output based on the new document.

In LangChain, you can use `RefineDocumentsChain` as part of the load_summarize_chain method. What you need to do is setting `refine` as `chain_type` of your chain.

### Prompt design with `Refine` chain

With LangChain, the `refine` chain requires two prompts.

The question prompt to generate the output for subsequent task. The refine prompt to refine the output based on the generated content.

In this example, the question prompt is:

```
Please provide a summary of the following text.
TEXT: {text}
SUMMARY:
```

and the refine prompt is:

```
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```{text}```
BULLET POINT SUMMARY:
```


In [18]:
question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

### Generate summaries using Refine method

After you define prompts, you initiate a summarization chain using `refine` chain type.

In [19]:
refine_chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

Then, you use the summarization chain to summarize document using Refine method.

In [20]:
refine_outputs = refine_chain({"input_documents": pages})

Below you can see the resulting summaries.

In [21]:
final_refine_data = []
for doc, out in zip(
    refine_outputs["input_documents"], refine_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_refine_data.append(output)

In [22]:
pdf_refine_summary = pd.DataFrame.from_dict(final_refine_data)
pdf_refine_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_refine_summary.reset_index(inplace=True, drop=True)
pdf_refine_summary.head()

,file_name,file_type,page_number,chunks,concise_summary
0,2. Nutraceuticals research,.pdf,0,REVIEW\nNutraceuticals: opening the debate for...,Here is a summary of the text with the main po...
1,2. Nutraceuticals research,.pdf,1,Introduction\nThere is a growing overlap betwe...,Here is a summary of the text with the main po...
2,2. Nutraceuticals research,.pdf,1,by clinical studies; (v) an evaluation of poss...,"Here is a summary of the text, including the m..."
3,2. Nutraceuticals research,.pdf,2,Table 1\nSome deﬁnitions for food-derived prod...,"Here is a summary of the text, including the m..."
4,2. Nutraceuticals research,.pdf,2,(FUFOSE). Scientiﬁc concepts of\nfunctional fo...,Here is a summary of the provided text:\n\nThe...


In [23]:
index = 3
print("[Context]")
print(pdf_refine_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_refine_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_refine_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_refine_summary["file_name"].iloc[index])

[Context]
Table 1
Some deﬁnitions for food-derived products
Food supplement (United States
Government Ofﬁce, 1994)
A product (other than tobacco) in the form of a
capsule, powder, softgel or gelcap intended to
supplement the diet to enhance health that
bears or contains one or more of the
following dietary ingredients: a vitamin,
mineral, amino acid, or other botanical
or dietary substance.
United States Food and Drug Administration
(FDA). Dietary Supplement Health and Education
Act (DSHEA). U.S. Department of Health and
Human Services. 1994. United States. Public
Law 103–417, available at FDA Website:
http://www.fda.gov. [42]
Food supplement (EU, 2002)
Food product whose purpose is to
supplement the normal diet and which
consists of a concentrated source of nutrients
or other substances with nutritional effects
or physiological, single or in combination,
marketed in dosed formulations, such as capsules,
tablets, tablets or pills, designed to be taken
in small individual quantities mea

### Considerations

In short, the Refine method for text summarization with LLMs can pull in more relevant context and may be less lossy than Map Reduce. However, it requires many more calls to the LLM than Stuffing, and these calls are not independent, meaning they cannot be parallelized. Additionally, there is some potential dependency on the ordering of the documents. Latest documents they might become more relevant as this method suffers from recency bias.

## Conclusion


In this notebook you learn about different techniques to summarize long documents with LangChain and PaLM API. What you have seen in this notebook are only some of the possibilities you have. For example, there is another method called the Map-Rerank method which involves running an initial prompt on each chunk of data, which not only tries to complete a task but also gives a score for how certain it is in its answer. The responses are then ranked according to this score, and the highest score is returned.

With that being said, it is important to highlight that depending on your needs you may consider to use pure Foundational model with a custom framework to build generative ai application.

Here are some of the benefits of using a foundational model with a custom framework:

 - More flexibility to implement your application with different LLMs, prompting templates, document handling strategies and more.

 - More control to customize your generative applications based on your scenario.

 - Better performance to improve latency and scalability of your application.
